Спарсенный датасет можно скачать отсюда: https://www.kaggle.com/theovall/phonereviews

### Загрузка и обработка датасета

In [1]:
# Базовые библиотеки
import numpy as np
import pandas as pd

In [2]:
# Загружаем датасет
train_data = pd.read_csv('data.csv').dropna().drop_duplicates(ignore_index = True)
train_data.head()

,Review,Rating
0,3D Touch просто восхитительная вещь! Заряд дер...,5
1,"Отключается при температуре близкой к нулю, не...",4
2,"В Apple окончательно решили не заморачиваться,...",3
3,Постарался наиболее ёмко и коротко описать все...,4
4,Достойный телефон. Пользоваться одно удовольст...,5


In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321101 entries, 0 to 321100
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Review  321101 non-null  object
 1   Rating  321101 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 4.9+ MB


In [4]:
# Удалим отзывы с оценкой 3, чтобы снизить неопределенность
train_data = train_data[train_data['Rating'] != 3]

In [5]:
# 1 - позитивный, 0 - негативный
convert_dict = {1: 0, 2: 0, 4: 1, 5: 1}
train_data['Label'] = train_data['Rating'].map(convert_dict)

In [6]:
# Баланс классов
print('Доля положительных отзывов:', 
      np.round(len(train_data[train_data['Label'] == 1]) / len(train_data), 2))
print('Доля отрицательных отзывов:', 
      np.round(len(train_data[train_data['Label'] == 0]) / len(train_data), 2))

Доля положительных отзывов: 0.83
Доля отрицательных отзывов: 0.17


In [7]:
# Аномальные рейтинги
print('Оценки пользователей', train_data['Rating'].unique())
print('Количество странных объектов', len(train_data[(train_data['Rating'] == 0) | (train_data['Rating'] == 7) | (train_data['Rating'] == 9)]))

Оценки пользователей [5 4 2 1 7 0 9]
Количество странных объектов 3


In [8]:
# Удалим безболезненно
ind_drop = train_data[(train_data['Rating'] == 0) | (train_data['Rating'] == 7) | (train_data['Rating'] == 9)].index
train_data = train_data.drop(ind_drop).reset_index(drop = True)

### Бейзлайны (LinearSVC и LogisticRegression)

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [10]:
# Сразу выставляем баланс классов согласно их весу в выборке
logreg = LogisticRegression(max_iter = 1000, n_jobs = -1, class_weight = 'balanced')
svc = LinearSVC(max_iter = 3000, class_weight = 'balanced')

# Используем биграммы и ограничиваем количество признаков для ускорения вычислений
vectorizer = CountVectorizer(ngram_range = (1, 2), max_features = 15000)
tfidf = TfidfVectorizer(ngram_range = (1, 2), max_features = 15000)

# Пайплайны для алгоритмов
pipe_logreg = Pipeline([('vec', vectorizer),
                        ('clf', logreg)])
pipe_svc = Pipeline([('vec', tfidf),
                     ('clf', svc)])

In [11]:
%%time
score_logreg = cross_val_score(pipe_logreg, train_data['Review'], train_data['Label'],
                               scoring = 'accuracy', cv = 3, n_jobs = -1)
print('Mean accuracy logreg:', np.round(score_logreg.mean(), 4))

Mean accuracy logreg: 0.8976
Wall time: 2min 48s


In [12]:
%%time
score_svc = cross_val_score(pipe_svc, train_data['Review'], train_data['Label'],
                            scoring = 'accuracy', cv = 3, n_jobs = -1)
print('Mean accuracy svc:', np.round(score_svc.mean(), 4))

Mean accuracy svc: 0.8964
Wall time: 1min 51s


Значимой разницы в качестве нет, оставляем SVM, т.к. считает быстрее

### Автоанализатор тональности как дополнительный признак

In [14]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [15]:
# Берем уверенность модели в положительности отзыва
results = model.predict(train_data['Review'], k=5)
train_data['Sentiment'] = [result['positive'] for result in results]
train_data.head()

,Review,Rating,Label,Sentiment
0,3D Touch просто восхитительная вещь! Заряд дер...,5,1.0,0.787941
1,"Отключается при температуре близкой к нулю, не...",4,1.0,0.136618
2,Постарался наиболее ёмко и коротко описать все...,4,1.0,0.050341
3,Достойный телефон. Пользоваться одно удовольст...,5,1.0,0.546748
4,6s gold 64gb,5,1.0,0.029322


In [17]:
# Проверяем качество модели с новым признаков
svc = LinearSVC(max_iter = 3000, class_weight = 'balanced')
tfidf = TfidfVectorizer(ngram_range = (1, 2), max_features = 15000)

coltrans = ColumnTransformer([('vec', tfidf, 'Review')],
                             remainder = 'passthrough')

feature_pipe = Pipeline([('trans', coltrans),
                         ('clf', svc)])

score = cross_val_score(feature_pipe, train_data[['Review', 'Sentiment']], 
                        train_data['Label'], cv = 3, scoring = 'accuracy', n_jobs = -1)

In [18]:
print('Mean accuracy svc + analyzer:', np.round(score.mean(), 4))

Mean accuracy svc + analyzer: 0.8964


Прироста в качестве нет, убираем признак

### Поиск параметров по сетке

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [20]:
# Делим выборку на 2 части: одна для train/test, вторая - валидационная
X_train, X_test, y_train, y_test = train_test_split(train_data['Review'],
                                                    train_data['Label'],
                                                    stratify = train_data['Label'],
                                                    test_size = 0.3,
                                                    random_state = 42)

In [21]:
svc = LinearSVC(max_iter = 3000, class_weight = 'balanced')
tfidf = TfidfVectorizer(max_features = 30000) # оставим больше информации в модели

pipe = Pipeline([('vec', tfidf),
                 ('clf', svc)])
param_grid = {'vec__ngram_range': [(1, 2), (1, 3)],
              'clf__C': [1, 0.1, 0.01]}
model = GridSearchCV(pipe, param_grid = param_grid, cv = 3,
                     scoring = 'accuracy', n_jobs = -1).fit(X_train, y_train)

In [28]:
print('Best params:', model.best_params_)
print('Best accuracy:', np.round(model.best_score_, 4))

Best params: {'clf__C': 1, 'vec__ngram_range': (1, 3)}
Best accuracy: 0.9132


In [29]:
# Проверка на переобучение на валидационной выборке
print('Test accuracy', np.round(model.best_estimator_.score(X_test, y_test), 4))

Test accuracy 0.9133


### Лучшая модель

In [26]:
best_pipe = Pipeline([('vec', TfidfVectorizer(max_features = 30000, ngram_range = (1, 3))),
                      ('clf', LinearSVC(max_iter = 3000, class_weight = 'balanced'))])
fitted_model = best_pipe.fit(train_data['Review'], train_data['Label'])

In [27]:
# Сохраняем модель
from joblib import dump
dump(fitted_model, 'model.joblib')

['model.joblib']